In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 加载乳腺癌数据集
data = load_breast_cancer()
X, y = data.data, data.target

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 数据标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc

# 定义激活函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def leaky_relu(x):
    return np.where(x > 0, x, 0.01 * x)

def swish(x):
    return x * sigmoid(x)

# 自定义激活函数的 MLPClassifier
class CustomMLPClassifier(MLPClassifier):
    def __init__(self, activation='relu', hidden_layer_sizes=(100,), **kwargs):
        super().__init__(activation=activation, hidden_layer_sizes=hidden_layer_sizes, **kwargs)
        self.custom_activation = activation

    def _forward_pass(self, X):
        activations = [X]
        for i in range(len(self.coefs_)):
            hidden_activation = activations[-1] @ self.coefs_[i] + self.intercepts_[i]
            if self.custom_activation == 'leaky_relu':
                activations.append(leaky_relu(hidden_activation))
            elif self.custom_activation == 'swish':
                activations.append(swish(hidden_activation))
            else:
                activations.append(super().activation(hidden_activation))
        return activations[-1]

    def predict(self, X):
        activations = self._forward_pass(X)
        if self.out_activation_ == 'softmax':
            return np.argmax(activations, axis=1)
        else:
            return activations > 0.5

# 数据加载与预处理
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 激活函数列表
activations = ['relu', 'tanh', 'logistic', 'leaky_relu', 'swish']

# 训练模型并评估性能
results = []
for activation in activations:
    model = CustomMLPClassifier(activation=activation, hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    y_scores = model.predict_proba(X_test_scaled)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_scores)
    roc_auc = auc(fpr, tpr)
    results.append({
        'activation': activation,
        'accuracy': accuracy_score(y_test, y_pred),
        'roc_auc': roc_auc,
        'fpr': fpr,
        'tpr': tpr
    })
    print(f"Activation: {activation}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print()

# 绘制 ROC 曲线
plt.figure(figsize=(10, 8))
for result in results:
    plt.plot(result['fpr'], result['tpr'], label=f"{result['activation']} (AUC = {result['roc_auc']:.2f})")
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Different Activation Functions')
plt.legend(loc='lower right')
plt.grid()
plt.show()

# 绘制激活函数图像
x = np.linspace(-10, 10, 1000)
plt.figure(figsize=(10, 8))
for activation in activations:
    if activation == 'relu':
        y = np.maximum(x, 0)
    elif activation == 'tanh':
        y = np.tanh(x)
    elif activation == 'logistic':
        y = sigmoid(x)
    elif activation == 'leaky_relu':
        y = leaky_relu(x)
    elif activation == 'swish':
        y = swish(x)
    plt.plot(x, y, label=activation)
plt.xlabel('x')
plt.ylabel('Activation')
plt.title('Activation Functions')
plt.legend(loc='upper right')
plt.grid()
plt.show()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.